# Notebook — Procesamiento de `data_latam_countries_coafina_hackathon.xlsx`
Este notebook contiene el código para **limpiar y normalizar** las pestañas del archivo Excel `data_latam_countries_coafina_hackathon.xlsx` y generar CSVs listos para unir con los archivos `e-waste-year_fixed.csv` (2018-2022 u otros).

**Salida esperada**: carpeta `processed_excel/` con los CSVs:
- `overall_data_clean.csv`
- `ewaste_generated_per_year_long.csv`
- `eee_pom_per_year_long.csv`
- `share_eee_categories_categories_long.csv`
- `share_categories_ewaste_generat_categories_long.csv`
- `metadata.json`

> Se deben ejecutar las celdas en orden. El notebook está diseñado para usarse en el entorno donde están los archivos.

In [22]:
# Imports y rutas
import pandas as pd
import re, json
from pathlib import Path
from datetime import datetime

BASE = Path("../data/raw")
EXCEL_PATH = BASE / "data_latam_countries_coafina_hackathon.xlsx"
OUT_DIR = BASE / "processed_excel"
OUT_DIR.mkdir(exist_ok=True, parents=True)

print("Excel path:", EXCEL_PATH)
print("Output dir:", OUT_DIR)

Excel path: ..\data\raw\data_latam_countries_coafina_hackathon.xlsx
Output dir: ..\data\raw\processed_excel


In [23]:
# Funciones helper: limpieza de nombres, numeric conversion y normalización de país
import pandas as pd
import re

def clean_colname(col: str) -> str:
    s = str(col)
    s = s.strip()
    s = s.replace("(", "_").replace(")", "_")
    s = re.sub(r"[%\-/\.\s]+", "_", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"[^\w_]", "", s)
    s = s.strip("_").lower()
    return s

def to_numeric_series(series: pd.Series) -> pd.Series:
    # eliminar comas y espacios y convertir a numérico (NaN si no convertible)
    s2 = series.astype(str).str.replace(r"[,\s]+", "", regex=True)
    s2 = s2.replace({"": pd.NA, "nan": pd.NA, "None": pd.NA})
    return pd.to_numeric(s2, errors="coerce")

def standardize_country_name(series: pd.Series) -> pd.Series:
    return series.astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.title()

print('Funciones definidas.')

Funciones definidas.


In [24]:
# Función para convertir hojas ancho->largo buscando columnas-año
def melt_year_sheet(df: pd.DataFrame, value_name_pref="value"):
    # detecta columnas que contienen años (19xx o 20xx)
    year_cols = [c for c in df.columns if re.search(r"(19|20)\d{2}", c)]
    if not year_cols:
        return None
    id_vars = [c for c in df.columns if c not in year_cols]
    # mapear cada columna a su año
    def extract_year(cname):
        m = re.search(r"(19|20)\d{2}", cname)
        return int(m.group(0)) if m else None
    year_map = {c: extract_year(c) for c in year_cols}
    df_long = df.melt(id_vars=id_vars, value_vars=year_cols, var_name="year_col", value_name=value_name_pref)
    df_long["year"] = df_long["year_col"].map(year_map)
    df_long = df_long.drop(columns=["year_col"])
    # convertir valor
    df_long[value_name_pref] = to_numeric_series(df_long[value_name_pref])
    if "country" in df_long.columns:
        df_long["country"] = standardize_country_name(df_long["country"])
    # reordenar columnas para claridad
    cols_order = ["country", "year"] + [c for c in df_long.columns if c not in ("country","year")]
    cols_order = [c for c in cols_order if c in df_long.columns]
    return df_long[cols_order]

print('melt_year_sheet definida.')

melt_year_sheet definida.


In [25]:
# Lectura del Excel, normalización de columnas, y procesamiento por hoja
xl = pd.ExcelFile(EXCEL_PATH)
sheets = xl.sheet_names
print('Pestañas encontradas:', sheets)

cleaned = {}
for sheet in sheets:
    df = xl.parse(sheet)
    df.columns = [clean_colname(c) for c in df.columns]
    # asegurar columna 'country'
    if "country" not in df.columns:
        candidates = [c for c in df.columns if "country" in c or "pais" in c or "país" in c or "country_name" in c]
        if candidates:
            df = df.rename(columns={candidates[0]: "country"})
    if "country" in df.columns:
        df["country"] = standardize_country_name(df["country"])
    cleaned[sheet] = df.copy()

outputs = {}
for sheet, df in cleaned.items():
    print('\nProcesando hoja:', sheet)
    if sheet == "overall_data":
        cols = df.columns.tolist()
        keep = []
        keywords = ["country", "gdp", "household", "ewaste_management_collection_rate", "ewaste_generated", "eee_placed_on_market", "ewaste_formally_collected", "generated_ewaste_plastic", "generated_bfrs_from_ewaste"]
        for kw in keywords:
            matches = [c for c in cols if kw in c]
            if matches:
                for m in matches:
                    if m not in keep:
                        keep.append(m)
        if "country" in cols and "country" not in keep:
            keep = ["country"] + keep
        overall = df[keep].copy() if keep else df.copy()
        # convertir a numérico donde aplique
        for c in overall.columns:
            if c != "country":
                overall[c] = to_numeric_series(overall[c])
        outp = OUT_DIR / "overall_data_clean.csv"
        overall.to_csv(outp, index=False)
        outputs["overall_data_clean"] = str(outp)
        display(overall.head(20))
    elif re.search(r"year", sheet) or re.search(r"_per_year", sheet) or re.search(r"per_year", sheet):
        long = melt_year_sheet(df, value_name_pref="value")
        if long is not None:
            fname = OUT_DIR / f"{sheet}_long.csv"
            long.to_csv(fname, index=False)
            outputs[f"{sheet}_long"] = str(fname)
            display(long.head(20))
        else:
            fname = OUT_DIR / f"{sheet}_raw.csv"
            df.to_csv(fname, index=False)
            outputs[f"{sheet}_raw"] = str(fname)
            display(df.head(10))
    elif re.search(r"share", sheet) or re.search(r"categories", sheet):
        if "country" in df.columns:
            melted = df.melt(id_vars=["country"], var_name="category", value_name="share_value")
            melted["share_value"] = to_numeric_series(melted["share_value"])
            melted["country"] = standardize_country_name(melted["country"])
            fname = OUT_DIR / f"{sheet}_categories_long.csv"
            melted.to_csv(fname, index=False)
            outputs[f"{sheet}_categories_long"] = str(fname)
            display(melted.head(20))
        else:
            fname = OUT_DIR / f"{sheet}_raw.csv"
            df.to_csv(fname, index=False)
            outputs[f"{sheet}_raw"] = str(fname)
            display(df.head(10))
    else:
        fname = OUT_DIR / f"{sheet}_raw.csv"
        df.to_csv(fname, index=False)
        outputs[f"{sheet}_raw"] = str(fname)
        display(df.head(10))

print('\nProcesamiento completado. Archivos generados:')
for k,v in outputs.items():
    print('-', k, '->', v)

Pestañas encontradas: ['overall_data', 'ewaste_generated_per_year', 'eee_pom_per_year', 'share_eee_categories', 'share_categories_ewaste_generat']

Procesando hoja: overall_data


,country,gdp_per_capita,average_household_size,ewaste_management_collection_rate,ewaste_generated_kg_inh,eee_placed_on_market_kg_inh,ewaste_formally_collected_kg_inh,generated_ewaste_plastic_kg_inh,generated_bfrs_from_ewaste_kg_inh
0,Argentina,23040.00,3.3,0.040,7.7,7.0,0.30,1.9,0.2
1,Bolivia,9110.00,3.5,0.040,4.7,7.5,0.20,1.2,0.1
2,Chile,25154.00,3.6,0.050,7.9,13.4,0.40,2.1,0.2
3,Costa_Rica,20443.00,3.2,0.080,13.2,15.4,1.00,4.1,0.4
4,Ecuador,11878.00,3.8,0.040,5.1,7.5,0.20,1.7,0.2
5,El_Salvador,9164.00,4.1,0.010,5.0,7.5,0.10,1.5,0.1
6,Guatemala,17667.00,4.8,0.000,2.9,4.7,NaN,0.9,0.1
7,Honduras,5981.00,3.9,0.010,2.6,2.9,0.10,0.8,0.6
8,Nicaragua,1905.26,5.0,0.004,2.5,3.2,0.01,0.8,0.1
9,Panama,32850.00,3.7,0.002,8.6,15.2,0.02,2.5,0.2



Procesando hoja: ewaste_generated_per_year


,country,year,value
0,Argentina,2009,5.8
1,Bolivia,2009,2.4
2,Chile,2009,3.9
3,Costa_Rica,2009,9.0
4,Ecuador,2009,2.8
5,El_Salvador,2009,3.9
6,Guatemala,2009,1.8
7,Honduras,2009,1.8
8,Nicaragua,2009,1.6
9,Panama,2009,5.9



Procesando hoja: eee_pom_per_year


...


Procesando hoja: share_eee_categories


,country,category,share_value
0,Argentina,temperature_exchange_equipment,0.22
1,Bolivia,temperature_exchange_equipment,0.20
2,Chile,temperature_exchange_equipment,0.34
3,Costa_Rica,temperature_exchange_equipment,0.22
4,Ecuador,temperature_exchange_equipment,0.25
5,El_Salvador,temperature_exchange_equipment,0.28
6,Guatemala,temperature_exchange_equipment,0.22
7,Honduras,temperature_exchange_equipment,0.23
8,Nicaragua,temperature_exchange_equipment,0.31
9,Panama,temperature_exchange_equipment,0.24



Procesando hoja: share_categories_ewaste_generat


,country,category,share_value
0,Argentina,temperature_exchange_equipment,0.22
1,Bolivia,temperature_exchange_equipment,0.15
2,Chile,temperature_exchange_equipment,0.26
3,Costa_Rica,temperature_exchange_equipment,0.17
4,Ecuador,temperature_exchange_equipment,0.16
5,El_Salvador,temperature_exchange_equipment,0.24
6,Guatemala,temperature_exchange_equipment,0.17
7,Honduras,temperature_exchange_equipment,0.28
8,Nicaragua,temperature_exchange_equipment,0.26
9,Panama,temperature_exchange_equipment,0.24



Procesamiento completado. Archivos generados:
- overall_data_clean -> ..\data\raw\processed_excel\overall_data_clean.csv
- ewaste_generated_per_year_long -> ..\data\raw\processed_excel\ewaste_generated_per_year_long.csv
- eee_pom_per_year_long -> ..\data\raw\processed_excel\eee_pom_per_year_long.csv
- share_eee_categories_categories_long -> ..\data\raw\processed_excel\share_eee_categories_categories_long.csv
- share_categories_ewaste_generat_categories_long -> ..\data\raw\processed_excel\share_categories_ewaste_generat_categories_long.csv


In [27]:
import pandas as pd
import unicodedata
import re
from pathlib import Path

# --- 1) RUTAS ---
INPUT_DIR = Path("../data/processed_excel")
OUTPUT_DIR = Path("../data/processed_excel")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

FILES = [
    "eee_pom_per_year_long.csv",
    "ewaste_generated_per_year_long.csv",
    "overall_data_clean.csv",
    "share_categories_ewaste_generat_categories_long.csv",
    "share_eee_categories_categories_long.csv"
]

# --- 2) LIMPIEZA DE NOMBRES ---
def strip_accents(s):
    s = unicodedata.normalize("NFKD", str(s))
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def clean_name(s):
    s = strip_accents(s).lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^a-zA-Z\s]", "", s)
    return s

# --- 3) MAPEO MANUAL LATAM → ISO3 ---
manual_iso_map = {
    "colombia": "COL",
    "argentina": "ARG",
    "bolivia": "BOL",
    "bolivia plurinational state of": "BOL",
    "brazil": "BRA",
    "brasil": "BRA",
    "chile": "CHL",
    "chile republic of": "CHL",
    "costa rica": "CRI",
    "ecuador": "ECU",
    "el salvador": "SLV",
    "guatemala": "GTM",
    "honduras": "HND",
    "mexico": "MEX",
    "nicaragua": "NIC",
    "panama": "PAN",
    "paraguay": "PRY",
    "peru": "PER",
    "uruguay": "URY",
    "venezuela": "VEN",
    "haiti": "HTI",
    "dominican republic": "DOM",
    "republica dominicana": "DOM"
}

# --- 4) PROCESAR ARCHIVOS ---
for file in FILES:
    print(f"\nProcesando: {file}")
    df = pd.read_csv(INPUT_DIR / file)

    # Detectar columna país
    country_col = None
    for c in df.columns:
        if "country" in c.lower() or "pais" in c.lower():
            country_col = c
            break

    if country_col is None:
        print("⚠️  No se encontró columna de país — se omite este archivo.")
        continue

    # Normalizar país
    df["country_clean"] = df[country_col].astype(str).apply(clean_name)

    # Mapear ISO3
    df["iso3"] = df["country_clean"].map(manual_iso_map)

    # Guardar archivo procesado
    output_path = OUTPUT_DIR / file
    df.to_csv(output_path, index=False)
    print(f"✅ Guardado con ISO3 → {output_path}")

print("\n🎯 Proceso completado. Todos los archivos procesados están en /data/processed_csv_iso")



Procesando: eee_pom_per_year_long.csv
✅ Guardado con ISO3 → ..\data\processed_excel\eee_pom_per_year_long.csv

Procesando: ewaste_generated_per_year_long.csv
✅ Guardado con ISO3 → ..\data\processed_excel\ewaste_generated_per_year_long.csv

Procesando: overall_data_clean.csv
✅ Guardado con ISO3 → ..\data\processed_excel\overall_data_clean.csv

Procesando: share_categories_ewaste_generat_categories_long.csv
✅ Guardado con ISO3 → ..\data\processed_excel\share_categories_ewaste_generat_categories_long.csv

Procesando: share_eee_categories_categories_long.csv
✅ Guardado con ISO3 → ..\data\processed_excel\share_eee_categories_categories_long.csv

🎯 Proceso completado. Todos los archivos procesados están en /data/processed_csv_iso


In [28]:
# Guardar metadata del procesamiento en processed_excel/metadata.json
metadata = {
    "source_file": str(EXCEL_PATH),
    "processed_at": datetime.utcnow().isoformat() + "Z",
    "sheets_processed": list(cleaned.keys()),
    "outputs": outputs
}
with open(OUT_DIR / "metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)
print("metadata.json guardado en:", OUT_DIR / "metadata.json")

C:\Users\fabia\AppData\Local\Temp\ipykernel_8416\479035382.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "processed_at": datetime.utcnow().isoformat() + "Z",


FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\raw\\processed_excel\\metadata.json'